In [1]:
import torch

In [2]:
word = 'hellowworld'
sorted(word)

['d', 'e', 'h', 'l', 'l', 'l', 'o', 'o', 'r', 'w', 'w']

In [3]:
word2idx = {
    "d" : 0,
    "e": 1,
    "h": 2,
    "l": 3,
    "o": 4,
    "r": 5,
    "w": 6
}

idx2word = {}
for w in word2idx:
    idx2word[word2idx[w]] = w

def encode(seq):
    return [word2idx[w] for w in seq]

def decode(token):
    return [idx2word[i] for i in token]

def one_hot(idx, len = len(word2idx)):
    hots = []
    for i in range(len):
        if(i == idx):
            hots.append(1)
        else:
            hots.append(0)
    return hots

def batch_one_hot(seq):
    token = encode(seq)
    return [one_hot(i) for i in token]


In [4]:
input_size = 7
seq_len = 10
hidden_size = 7
batch_size = 1
num_layers = 1

In [5]:
input_seq = "helloworld"
output_seq ="worldhello"
x_data = batch_one_hot(input_seq)
y_data = batch_one_hot(output_seq)

inputs = torch.tensor(x_data).reshape(seq_len, batch_size, input_size).float()
labels = torch.tensor(y_data).reshape(-1, input_size).float()
print(inputs.shape)
print(labels.shape)

torch.Size([10, 1, 7])
torch.Size([10, 7])


In [6]:
class Model(torch.nn.Module):
    def __init__(self, input_size, hidden_size, batch_size, num_layers):
         super(Model, self).__init__()
         self.input_size = input_size
         self.hidden_size = hidden_size
         self.batch_size = batch_size
         self.num_layers = num_layers
         self.rnn = torch.nn.RNN(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=num_layers)

    def forward(self, inputs):
        init_hidden = torch.zeros(self.num_layers, self.batch_size, self.hidden_size)
        outputs, _ = self.rnn(inputs, init_hidden)
        return outputs.view(-1, self.hidden_size)

net = Model(input_size, hidden_size, batch_size, num_layers)

In [8]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.05)

for epoch in range(15):
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    _, idx = outputs.max(dim=1)
    idx = idx.data.numpy()
    print("Predicted string: ","".join(idx2word[x] for x in idx), end="")
    print(", Epoch [%d/15] loss=%.3f" % (epoch + 1, loss.item()))